> 所有的深度学习框架（torch、tf、mxnet 等）都是对矩阵微分的封装；

- references
    - https://atmos.washington.edu/~dennis/MatrixCalculus.pdf
    - https://math.stackexchange.com/questions/312077/differentiate-fx-xtax
- 注意识别和区分一些概念：
    - 从线性代数到矩阵分析：就是多了矩阵微积分的操作；
    - 标量/矢量关于矢量/矩阵的微分
        - 标量关于矢量
        - 矢量关于矢量
        - **标量关于矩阵**
        - **矢量关于矩阵**
    - scalar (loss) w.r.t sth 的导数，其 shape 就与 sth 的 shape 一致；
- 偏数学的推导及证明还是要回到代码，回到 torch 中的应用，尤其是 `loss.backward`
    - 数学证明里边一个很重要的技巧就是等价替换，也复用已知结论；

## $\mathbf y=\mathbf {Ax}$

- $\mathbf x\in \mathbb R^n, \mathbf A\in \mathbb R^{m\times n} \rightarrow \mathbf y\in \mathbb R^{m}$
- 线性变换的角度就是 $\mathbb R^n\rightarrow \mathbb R^m$ 的映射/投影（project）
    - transformer 中的 ffn（h -> 4h -> h)

$$
\mathbf{y} = \psi(\mathbf{x}),
$$


$$
\begin{equation}
\frac{\partial \mathbf{y}}{\partial \mathbf{x}} = 
\begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} & \cdots & \frac{\partial y_1}{\partial x_n} \\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} & \cdots & \frac{\partial y_2}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial y_m}{\partial x_1} & \frac{\partial y_m}{\partial x_2} & \cdots & \frac{\partial y_m}{\partial x_n}
\end{bmatrix}
\end{equation}
$$

- $\mathbf{y} = \psi(\mathbf{x}),$ 比如 $\mathbf y=\mathbf {Ax}$
- $\frac{\partial \mathbf y}{\partial \mathbf x}$ 向量（多元输出，multi-variables）对向量（多元输入，multi-inputs）的导数，此时的 gradient 是 jacobian matrix


$$
\begin{split}
&\mathbf y=\mathbf {Ax}\\
&\frac{\partial \mathbf y}{\partial \mathbf x}=\mathbf A
\end{split}
$$

- 我们来进行简单的推导
    - $y_i=A_{[i]}x=\sum_k a_{ik}x_k$
        - $y_1=\sum_ka_{1k}x_k$
- 一个特例，当 $A$ 为一个行向量时（$\mathbf w^T$），退化为一个多元输入，单输出（标量 scalar 输出）的内积运算，此时的导数为与输入等shape的向量；

    $$
    y=\mathbf w^T\mathbf x
    $$
  
    - $y=\mathbf w^T\mathbf x=\sum_iw_ix_i$
 

$$
\frac{\partial y}{\partial \mathbf x}=\begin{bmatrix}w_1,w_2,\cdots,w_n\end{bmatrix}^T=\mathbf w
$$

In [9]:
import torch

# 定义矩阵 A 和向量 x
A = torch.tensor([[1.0, 2.0],
                  [3.0, 4.0],
                  [5.0, 6.0]], requires_grad=True)

x = torch.tensor([[0.5], [1.0]], requires_grad=True)

# 计算 y = A * x
y = torch.matmul(A, x)

# 初始化雅可比矩阵
jacobian = torch.zeros_like(A)

# 计算每个 y 的元素关于 x 的导数
for i in range(y.size(0)):
    y[i].backward(retain_graph=True)
    jacobian[i] = x.grad.view(1, -1)
    x.grad.zero_()

print(jacobian)

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]])


### $\mathbf y=\mathbf x \mathbf W$

- $\mathbf x$ 是一个行向量
- 对原式做等价替换：$\mathbf y^T=\mathbf W^T\mathbf x^T$

$$
\frac{\partial \mathbf y}{\partial \mathbf x}=\frac{\partial \mathbf y^T}{\partial \mathbf x^T}=\mathbf W^T
$$

构造一个简单的示例辅助理解：

$$
\begin{split}
\begin{pmatrix}y_1 & y_2 & y_3\end{pmatrix}&=\begin{pmatrix}x_1 & x_2\end{pmatrix}\begin{pmatrix}w_{11} & w_{12} & w_{13}\\
w_{21} & w_{22} & w_{23}
\end{pmatrix}\\
&=\begin{pmatrix}w_{11}x_1+w_{21}x_2 & w_{12}x_1+w_{22}x_2 & w_{13}x_1+w_{23}x_2\end{pmatrix}
\end{split}
$$

$$
\begin{pmatrix}\frac{\partial y_1}{\partial \mathbf x} & \frac{\partial y_2}{\partial \mathbf x} & \frac{\partial y_3}{\partial \mathbf x}\end{pmatrix}=\begin{pmatrix}w_{11} & w_{21}\\
w_{12} & w_{22}\\
w_{13} & w_{23}
\end{pmatrix}
$$

In [14]:
import torch

# 定义矩阵 A 和向量 x
A = torch.tensor([[1.0, 2.0],
                  [3.0, 4.0],
                  [5.0, 6.0]], requires_grad=True)

x = torch.tensor([[0.5, 1.0, 2]], requires_grad=True)

# 计算 y = A * x
y = torch.matmul(x, A)[0]

# 初始化雅可比矩阵
jacobian = torch.zeros(2, 3)

# 计算每个 y 的元素关于 x 的导数
for i in range(y.size(0)):
    y[i].backward(retain_graph=True)
    jacobian[i] = x.grad.view(1, -1)
    x.grad.zero_()

print(jacobian)

tensor([[1., 3., 5.],
        [2., 4., 6.]])


## $\alpha=\mathbf y^T\mathbf A\mathbf x$

$$
\begin{split}
&\frac{\partial \alpha}{\partial \mathbf x}=\mathbf y^T\mathbf A\\
&\frac{\partial \alpha}{\partial \mathbf y}=\mathbf x^T\mathbf A^T
\end{split}
$$

来看证明：
- 对于第一个导数
    - $\mathbf w^T=\mathbf y^T\mathbf A$
    - $\alpha=\mathbf w^T\mathbf x$
    - $\frac{\partial \alpha}{\partial \mathbf x}=\mathbf w^T=\mathbf y^T\mathbf A$
- 对于第二个导数
    - $\alpha=\alpha^T=\mathbf x^T\mathbf A^T\mathbf y$
    - $\frac{\partial \alpha}{\partial \mathbf y}=\mathbf x^T\mathbf A^T$

## $\alpha =\mathbf x^T\mathbf A\mathbf x$

$$
\frac{\partial \alpha}{\partial \mathbf x}=(\mathbf A+\mathbf A^T)\mathbf x
$$

证明，基于矩阵矢量乘法的定义/计算：

$$
\begin{split}
&\alpha=\sum_ix_i\sum_ja_{ij}x_j=\sum_i\sum_jx_ia_{ij}x_j\\
&\frac{\partial \alpha}{\partial x_k}=\sum_ix_ia_{ik}+\sum_jx_ka_{kj}\\
&\frac{\partial \alpha}{\partial \mathbf x}=\mathbf A^T\mathbf x+\mathbf A\mathbf x=(\mathbf A+\mathbf A^T)\mathbf x
\end{split}
$$

一些特例：

- $\mathbf A^T=\mathbf A$ 时，$\frac{\partial \alpha}{\partial \mathbf A}=2\mathbf A\mathbf x$

In [21]:
x = torch.randn(3, 1, requires_grad=True)
A = torch.randn(3, 3, requires_grad=True)
y = (x.T @ A) @ x
y.backward()
torch.allclose(x.grad, (A + A.T) @ x)

True

## $\mathbf y=\mathbf A\mathbf x$：关于矩阵求导

$$
\frac{\partial \mathbf y}{\partial \mathbf A}
$$

- 是一个三维的tensor
    - $\frac{\partial y_i}{\partial \mathbf A}$ 各是一个矩阵
    - $y_1=w_{11}x_1+w_{12}x_2$ => $\begin{bmatrix}x_1 & x_2\\0 & 0\end{bmatrix}$
    - $y_2=w_{21}x_1+w_{22}x_2$ => $\begin{bmatrix}0 & 0\\x_1 & x_2\end{bmatrix}$

https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html#optional-reading-tensor-gradients-and-jacobian-products

In [30]:
import torch

# 定义矩阵 A 和向量 x
A = torch.tensor([[1.0, 2.0],
                  [3.0, 4.0],
                  # [5.0, 6.0]
                 ], 
                 requires_grad=True)

x = torch.tensor([[0.5], [1.0]], requires_grad=True)

# 计算 y = A * x
y = torch.matmul(A, x)
print(y.shape)
# 计算 y 对 A 的雅可比矩阵
# v^T·J
y.backward(torch.ones_like(y))

# 获取雅可比矩阵
jacobian = A.grad
jacobian

torch.Size([2, 1])


tensor([[0.5000, 1.0000],
        [0.5000, 1.0000]])

In [31]:

# 定义矩阵 A 和向量 x
A = torch.tensor([[1.0, 2.0],
                  [3.0, 4.0],
                  [5.0, 6.0]
                 ], requires_grad=True)

x = torch.tensor([[0.5], [1.0]], requires_grad=True)

# 计算 y = A * x
y = torch.matmul(A, x)

# 初始化一个与 A 形状相同的零张量来存储雅可比矩阵
jacobian = torch.zeros((y.size(0), A.size(0), A.size(1)))

# 逐元素计算雅可比矩阵
for i in range(y.size(0)):
    # 清除梯度
    A.grad = None
    
    # 对 y 中的第 i 个元素进行反向传播
    y[i].backward(retain_graph=True)
    
    # 将计算得到的梯度存储在雅可比矩阵中
    jacobian[i] = A.grad
jacobian

tensor([[[0.5000, 1.0000],
         [0.0000, 0.0000]],

        [[0.0000, 0.0000],
         [0.5000, 1.0000]]])

In [26]:
# 定义矩阵 A 和向量 x
A = torch.tensor([[1.0, 2.0],
                  [3.0, 4.0],
                  [5.0, 6.0]], requires_grad=True)

x = torch.tensor([[0.5], [1.0]], requires_grad=True)

# 计算 y = A * x
y = torch.matmul(A, x)
print(y.shape)
# 计算 y 对 A 的雅可比矩阵
# v^T·J
y.backward(torch.tensor([1., 2., 3.]).view(-1, 1))

# 获取雅可比矩阵
jacobian = A.grad
jacobian

torch.Size([3, 1])


tensor([[0.5000, 1.0000],
        [1.0000, 2.0000],
        [1.5000, 3.0000]])

## loss backward

In [1]:
import torch

# 设定输入 x 和权重 W，b 为偏置
x = torch.tensor([[1.0, 2.0]], requires_grad=True)  # 1x2 行向量
W = torch.tensor([[0.5, -0.5], [1.5, -1.0]], requires_grad=True)  # 2x2 矩阵
b = torch.tensor([[0.1, -0.1]], requires_grad=True)  # 1x2 行向量

# 前向传播计算 z = xW + b
z = x @ W + b  # 矩阵乘法加上偏置

# 定义一个简单的标量损失函数，假设为 z 的和
L = z.sum()

# 进行反向传播计算梯度
L.backward()

# 打印梯度
print("dL/dx:", x.grad)
print("dL/dW:", W.grad)
print("dL/db:", b.grad)

# 手动验证梯度计算
dL_dz = torch.ones_like(z)  # 因为 L = z.sum(), dL/dz = 1
dz_dW = x.t()  # d(xW+b)/dW = x^T
manual_dL_dW = dz_dW @ dL_dz  # outer product
print("Manual dL/dW:", manual_dL_dW)

dL/dx: tensor([[0.0000, 0.5000]])
dL/dW: tensor([[1., 1.],
        [2., 2.]])
dL/db: tensor([[1., 1.]])
Manual dL/dW: tensor([[1., 1.],
        [2., 2.]], grad_fn=<MmBackward0>)


### Learning from data

In [22]:
from IPython.display import Image

In [29]:
Image(url='../../imgs/s34917042.jpg', width=400)

- 这本书包括 Strang 老师的 MIT 公开课，陪伴我读研以及工作直至今天；
- 整本书非常的凝练，有的放矢，面向现代的AI计算及应用；
    - learning from data （data-driven 的方式从数据中进行机器学习/深度学习/AI训练等）的观念就已经非常非常的现代了；
- 后边会做一期单独的视频导读这本书；